```mermaid
flowchart LR
    A[entrada X,y] --> B[Woodwork init]
    B --> C{overrides\nuser}
    C -->|force\ncategorical| D1
    C -->|force\nnumeric| D2
    D1 --> E[ID & alta-unicidade\nremovidas]
    D2 --> E
    E --> F{tipo}
    F -->|numérico| G[binning numérico: Optimal / Unsupervised]
    F -->|categórico| H[tratamento cat • encoding WoE ou freq]
    F -->|ignorado| I[(Ignora)]
    G --> J[refine_bins + checagens]
    H --> J
    J --> K[concat summaries]
    K --> L[pivot + PSI]
    L --> M[atributos finais :iv_, iv_dict_, schema_, …]
```

In [2]:
# imports
import os
import sys

# Adiciona o diretório raiz do projeto ao PYTHONPATH para importar o pacote local
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np
from nasabinning.binning_engine import NASABinner
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline

In [5]:
def generate_credit_risk_dataset(
    n: int = 1_000_000,
    seed: int = 42,
    anos_meses_possiveis: list[int] = [202301, 202302, 202303, 202304, 202305]
) -> pd.DataFrame:
    """
    Gera um DataFrame sintético para contexto de risco de crédito.
    
    Parâmetros
    ----------
    n : int
        Número de linhas (clientes) desejadas. Default = 1_000_000.
    seed : int
        Semente para aleatoriedade (reprodutibilidade). Default = 42.
    anos_meses_possiveis : list[int]
        Lista de códigos de referência de mês (AnoMesReferencia) utilizados
        para simular diferentes períodos. Ex.: [202301, 202302, ...].
    
    Retorna
    -------
    pd.DataFrame
        DataFrame com as seguintes colunas:
        - idade (float): idade simulada (média 40, desvio 10).
        - renda (float): renda mensal (média 3000, desvio 800).
        - score (float): score de crédito (média 600, desvio 100).
        - AnoMesReferencia (int): período em formato YYYYMM.
        - num_loans (int): número de empréstimos em aberto (distribuição Poisson).
        - debt_to_income (float): relação dívida / renda (entre 0 e 1).
        - num_credit_inquiries (int): número de consultas ao crédito (Poisson).
        - marital_status (categoria): estado civil.
        - education_level (categoria): nível de escolaridade.
        - employment_type (categoria): tipo de vínculo empregatício.
        - residential_status (categoria): situação de moradia.
        - default_90d (int): variável-alvo binária simulada (0 = sem default, 1 = default em até 90 dias).
    """
    np.random.seed(seed)
    
    # 1) Variáveis numéricas básicas
    idade = np.random.normal(loc=40, scale=10, size=n)
    renda = np.random.normal(loc=3000, scale=800, size=n)
    score = np.random.normal(loc=600, scale=100, size=n)
    
    # Garantir que renda e score não fiquem com valores negativos (se ocorrerem)
    renda = np.clip(renda, a_min=200, a_max=None)  # renda mínima de 200
    score = np.clip(score, a_min=300, a_max=850)   # score entre 300 e 850 (faixa típica)
    idade = np.clip(idade, a_min=18, a_max=90)      # faixa etária entre 18 e 90 anos
    
    # 2) Período de referência (AnoMesReferencia)
    AnoMesReferencia = np.random.choice(anos_meses_possiveis, size=n)
    
    # 3) Variáveis adicionais numéricas
    # Número de empréstimos em aberto (Poisson, média = 2)
    num_loans = np.random.poisson(lam=2, size=n)
    
    # Relação dívida / renda (uniforme entre 0 e 1, mas com tendência central)
    debt_to_income = np.random.beta(a=2, b=5, size=n)  # distrib. beta para dar assimetria
    
    # Número de consultas de crédito nos últimos 6 meses (Poisson, média = 1)
    num_credit_inquiries = np.random.poisson(lam=1, size=n)
    
    # 4) Variáveis categóricas
    # Estado civil
    marital_status = np.random.choice(
        ["Single", "Married", "Divorced", "Widowed"],
        size=n,
        p=[0.4, 0.45, 0.1, 0.05]
    )
    
    # Nível de escolaridade
    education_level = np.random.choice(
        ["High School", "Bachelor", "Master", "PhD", "Other"],
        size=n,
        p=[0.35, 0.4, 0.15, 0.05, 0.05]
    )
    
    # Tipo de emprego
    employment_type = np.random.choice(
        ["Salaried", "Self-employed", "Unemployed", "Retired"],
        size=n,
        p=[0.6, 0.15, 0.2, 0.05]
    )
    
    # Status residencial
    residential_status = np.random.choice(
        ["Rent", "Own", "Mortgage"],
        size=n,
        p=[0.5, 0.3, 0.2]
    )
    
    # 5) Montagem do DataFrame
    df = pd.DataFrame({
        "idade": idade,
        "renda": renda,
        "score": score,
        "AnoMesReferencia": AnoMesReferencia,
        "num_loans": num_loans,
        "debt_to_income": debt_to_income,
        "num_credit_inquiries": num_credit_inquiries,
        "marital_status": marital_status,
        "education_level": education_level,
        "employment_type": employment_type,
        "residential_status": residential_status,
    })
    
    # 6) Variável-alvo (default em até 90 dias)
    # Critério simplificado: maior chance de default se score baixo, renda baixa e dívida alta.
    # Calculamos uma pontuação de risco parcial e adicionamos componente aleatório.
    # Primeiro, normalizamos score e renda para 0–1:
    score_norm = (df["score"] - df["score"].min()) / (df["score"].max() - df["score"].min())
    renda_norm = (df["renda"] - df["renda"].min()) / (df["renda"].max() - df["renda"].min())
    
    # Uma “pontuação de risco” arbitrária (quanto maior, mais propenso a default)
    risk_score = (1 - score_norm) * 0.4 + (1 - renda_norm) * 0.3 + df["debt_to_income"] * 0.2 \
                 + (df["num_loans"] / (df["num_loans"].max() + 1)) * 0.1
    
    # Transformar em probabilidade pelo mapa logístico simplificado:
    # prob_default ∈ (0, 1), depois comparamos com uniforme para definir 0/1.
    prob_default = 1 / (1 + np.exp(-3 * (risk_score - 0.5)))
    
    default_90d = (np.random.rand(n) < prob_default).astype(int)
    df["default_90d"] = default_90d
    
    return df

df = generate_credit_risk_dataset(n=1_000_000, seed=42)
print(df.shape)
display(df.head())

(1000000, 12)


,idade,renda,score,AnoMesReferencia,num_loans,debt_to_income,num_credit_inquiries,marital_status,education_level,employment_type,residential_status,default_90d
0,44.967142,3135.337484,706.472241,202301,1,0.524953,2,Single,Master,Unemployed,Own,0
1,38.617357,2902.795875,550.328682,202304,0,0.093070,1,Divorced,High School,Salaried,Rent,0
2,46.476885,3925.300219,612.926020,202301,2,0.726887,2,Married,High School,Self-employed,Own,0
3,55.230299,3160.068631,452.030663,202304,1,0.385445,3,Married,Bachelor,Salaried,Rent,0
4,37.658466,3691.688555,569.015270,202305,2,0.300789,1,Married,High School,Salaried,Rent,1


In [4]:
df = pd.read_csv('../data/case_data_science_credit.csv', sep=';')
print(df.shape)
display(df.head())

(67463, 18)


,client_id,pf_ou_pj,grade,sub_grade,qtd_restritivos,verificacao_fonte_de_renda,razao_credito_tomado_vs_renda_informada,patrimonio_total,qtd_atrasos_ultimos_2a,valor_total_recuperacoes_ultimos_2a,contas_distintas_com_atraso,qtd_consultas_ultimos_6m,qtd_linhas_credito_abertas,saldo_rotativo_total,limite_rotativo_total,valor_total_emprestimos_tomados,taxa_juros_media_emprestimos_tomados,target
0,75521,PF,B,C4,0,Not Verified,16.284758,176346.62670,1,2.498291,0,0,13,24246,6619,10000,11.135007,0
1,28124,PF,C,D3,0,Source Verified,15.412409,39833.92100,0,2.377215,0,0,12,812,20885,3609,12.237563,0
2,8420,PF,F,D4,0,Source Verified,28.137619,91506.69105,0,4.316277,0,0,14,1843,26155,28276,12.545884,0
3,22553,PF,C,C3,0,Source Verified,18.043730,108286.57590,1,0.107020,0,0,7,13819,60214,11170,16.731201,0
4,62952,PF,C,D4,1,Source Verified,17.209886,44234.82545,1,1294.818751,0,3,13,1544,22579,16890,15.008300,0


In [6]:
binner_cat = NASABinner(strategy="categorical")
binner_cat.fit(df[["sub_grade"]], df["target"])
print(binner_cat._bin_summary_)

    variable  bin  count  event  non_event  event_rate
0  sub_grade    1   3250    498       2752    0.153231
1  sub_grade    2  64213   9071      55142    0.141264


In [8]:
binner_cat.get_bin_mapping("sub_grade")

,categoria,bin
0,NaN,-2
1,C4,1
2,D3,2
3,D4,3
4,C3,4
5,_RARE_,5
6,C5,6
7,A5,7
8,C2,8
9,B5,9


In [ ]:
binner = NASABinner(strategy="supervised", use_optuna=True)
binner.fit(df[["idade", "renda"]], df["default_90d"])   # roda sem TypeError

In [ ]:
print("IV:", round(binner.iv_, 4))
display(binner._bin_summary_.head(100))

# plotar estabilidade do modelo com Optuna

pivot_opt = binner.stability_over_time(
    X=df[["idade", "renda", "score", "debt_to_income", "AnoMesReferencia"]],
    y=df["default_90d"],
    time_col="AnoMesReferencia",
)

pivot_opt.to_excel('pivot_opt.xlsx')

# chama o wrapper da própria classe
binner.plot_event_rate_stability(
    pivot_opt,
    time_col_label="AnoMesReferencia",
    title_prefix="Estabilidade temporal - com Optuna"
)

In [ ]:
# usar NASABinner sem Optuna
binner = NASABinner(
    strategy="supervised",
    max_bins=5,
    min_event_rate_diff=0.01,
    monotonic=None,
    use_optuna=False,
    check_stability=True,
    force_categorical=["education_level",'employment_type'],
    force_numeric=["num_loans"],
)

# ainda nao consegue lidar com feature categorica
X = df[["idade", "renda", "score", "debt_to_income"]]
y = df["default_90d"]
binner.fit(X, y, time_col="AnoMesReferencia")

print("IV:", round(binner.iv_, 4))
display(binner._bin_summary_.head(100))

In [ ]:
binner._bin_summary_[binner._bin_summary_['variable']=='debt_to_income']

In [ ]:
# calcular pivot (já inclui a coluna AnoMesReferencia em X)
pivot = binner.stability_over_time(
    X=df[["idade", "renda", "score", "debt_to_income", "AnoMesReferencia"]],
    y=df["default_90d"],
    time_col="AnoMesReferencia",
)

pivot.to_excel('pivot.xlsx')

# chama o wrapper da própria classe
binner.plot_event_rate_stability(
    pivot,
    time_col_label="AnoMesReferencia",
)


In [ ]:
# # testar NASABinner COM Optuna
binner_opt = NASABinner(
    strategy="supervised",
    use_optuna=True,
    check_stability=True,
    monotonic="descending",
    strategy_kwargs={"n_trials": 15}
)

binner_opt.fit(X, y, time_col="AnoMesReferencia")
print("IV:", round(binner_opt.iv_, 4))
print("Melhores parâmetros:", binner_opt.best_params_)


In [ ]:
# plotar estabilidade do modelo com Optuna

pivot_opt = binner_opt.stability_over_time(
    X=df[["idade", "renda", "score", "debt_to_income", "AnoMesReferencia"]],
    y=df["default_90d"],
    time_col="AnoMesReferencia",
)

pivot_opt.to_excel('pivot_opt.xlsx')

# chama o wrapper da própria classe
binner_opt.plot_event_rate_stability(
    pivot_opt,
    time_col_label="AnoMesReferencia",
    title_prefix="Estabilidade temporal - com Optuna"
)